# Heart Failure Prediction and Recommendations
 Cardiovascular diseases, including heart failure, are a leading cause of mortality worldwide. Early and accurate prediction of patient risk, coupled with personalized treatment recommendations, is crucial for improving outcomes and reducing healthcare burdens. However, traditional risk assessment methods often lack precision, and generalized treatment guidelines may not be optimal for individual patients with unique characteristics.

 ## Project Aim
To develop a hybrid  system that accurately predicts the risk of heart failure in individual patients and provides personalized treatment recommendations, ultimately aiming to improve patient outcomes and reduce healthcare burdens associated with heart failure management.



# Finetuning T5

In [ ]:
!pip install transformers
!pip install torch

In [ ]:
import torch

# Check if a GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and being used.")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU instead.")

GPU is available and being used.


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

# Load model and tokenizer
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model_beforeLora = T5ForConditionalGeneration.from_pretrained(model_name)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
model_beforeLora

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:

#if we chose to use LOra, but not necessary due to flan t5 being lightweight
# LoRA configuration
lora_config = LoraConfig(
    task_type="CAUSAL_LM",  # Causal Language Modeling
    r=1,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q","k","v","o","wi_0","wi_1","wo","lm_head"] # these are the modules in the linear layers of T5
)

# Apply LoRA
model = get_peft_model(model_beforeLora, lora_config)


In [ ]:
model.print_trainable_parameters() #how many parameter we got

trainable params: 192,384 || all params: 77,153,536 || trainable%: 0.2494


In [ ]:
#send to gpu
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=384, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=1, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=1, out_features=384, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                

In [ ]:
import json
import pandas as pd
with open('/content/tailored_patient_data_new_chunk_2.json', 'r') as file:
    data = json.load(file)

# Flatten the data into input and target for easier finetuing on llms
flattened_data = []
for entry in data:
    row = {
        'input': f"Anemia: {entry['anemia']}, Age: {entry['age']}, Sex: {entry['sex']}, Creatinine: {entry['creatinine']}, "
                 f"Diabetes: {entry['diabetes']}, Ejection Fraction: {entry['ejection_fraction']}, "
                 f"High Blood Pressure: {entry['high_blood_pressure']}, Platelets: {entry['platelets']}, "
                 f"Serum Creatinine: {entry['serum_creatinine']}, Serum Sodium: {entry['serum_sodium']}, "
                 f"Smoking: {entry['smoking']}, Follow-up Time: {entry['follow_up_time']}, Dead: {entry['dead']}",
        'target': " ".join([r['recommendation'] for r in entry['recommendations']['all_recommendations_and_reasons']])
    }
    flattened_data.append(row)

df = pd.DataFrame(flattened_data)

In [ ]:
df

,input,target
0,"Anemia: 0, Age: 64, Sex: 1, Creatinine: 3230.2...",Consult nephrology for renal management and co...
1,"Anemia: 1, Age: 93, Sex: 0, Creatinine: 6053.7...",Consult nephrology for renal management and co...
2,"Anemia: 1, Age: 99, Sex: 1, Creatinine: 3376.7...",Start or optimize heart failure medications. C...
3,"Anemia: 1, Age: 83, Sex: 0, Creatinine: 4908.7...",Consult nephrology for renal management and co...
4,"Anemia: 1, Age: 16, Sex: 0, Creatinine: 4140.3...",Consult nephrology for renal management and co...
...,...,...
49995,"Anemia: 1, Age: 54, Sex: 0, Creatinine: 5542.5...",Consult nephrology for renal management and co...
49996,"Anemia: 0, Age: 72, Sex: 0, Creatinine: 1196.2...",Start or optimize heart failure medications. C...
49997,"Anemia: 1, Age: 35, Sex: 0, Creatinine: 5968.0...",Consult nephrology for renal management and co...
49998,"Anemia: 0, Age: 92, Sex: 1, Creatinine: 7245.1...",Consult nephrology for renal management and co...


In [ ]:
df.iloc[0]['input'] #whats in each input

'Anemia: 0, Age: 64, Sex: 1, Creatinine: 3230.2, Diabetes: 1, Ejection Fraction: 31.2, High Blood Pressure: 0, Platelets: 263119.9, Serum Creatinine: 5.3, Serum Sodium: 129.7, Smoking: 1, Follow-up Time: 15, Dead: 1'

In [ ]:
df2 = pd.DataFrame(flattened_data)
df2

,input,target
0,"Anemia: 0, Age: 64, Sex: 1, Creatinine: 3230.2...",Consult nephrology for renal management and co...
1,"Anemia: 1, Age: 93, Sex: 0, Creatinine: 6053.7...",Consult nephrology for renal management and co...
2,"Anemia: 1, Age: 99, Sex: 1, Creatinine: 3376.7...",Start or optimize heart failure medications. C...
3,"Anemia: 1, Age: 83, Sex: 0, Creatinine: 4908.7...",Consult nephrology for renal management and co...
4,"Anemia: 1, Age: 16, Sex: 0, Creatinine: 4140.3...",Consult nephrology for renal management and co...
...,...,...
49995,"Anemia: 1, Age: 54, Sex: 0, Creatinine: 5542.5...",Consult nephrology for renal management and co...
49996,"Anemia: 0, Age: 72, Sex: 0, Creatinine: 1196.2...",Start or optimize heart failure medications. C...
49997,"Anemia: 1, Age: 35, Sex: 0, Creatinine: 5968.0...",Consult nephrology for renal management and co...
49998,"Anemia: 0, Age: 92, Sex: 1, Creatinine: 7245.1...",Consult nephrology for renal management and co...


In [ ]:
df2.iloc[0]['target'] #whats in each target

'Consult nephrology for renal management and consider dialysis if symptoms of uremia develop. Carefully correct serum sodium under close monitoring. Initiate smoking cessation interventions. Optimize glycemic control with appropriate antidiabetic therapy. Discuss advanced heart failure therapies and palliative care options if prognosis worsens.'

In [ ]:
#to convert the words into numbers
def tokenize_function(examples):
    # Tokenize input and target text
    input_text = examples["input"]
    target_text = examples["target"]

    # Tokenize both input and target
    tokenized_input = tokenizer(input_text, padding="max_length", truncation=True, max_length=128)
    tokenized_target = tokenizer(target_text, padding="max_length", truncation=True, max_length=128)

    # Add input_ids and labels
    tokenized_input["labels"] = tokenized_target["input_ids"]
    return tokenized_input


In [ ]:
#convert to hugging face dataset for ease
!pip install datasets
from datasets import Dataset
dataset = Dataset.from_pandas(df2)


In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True) #aplly tokenization

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

Dataset({
    features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 50000
})

In [ ]:
#using only 10000 out of the 50000
train_dataset = tokenized_dataset.select(range(8000)) # 80% train
eval_dataset = tokenized_dataset.select(range(8000, 10000)) # 20% test

In [ ]:
#we dont need to send this data to gpu, its automatically done by the trainer
train_dataset, eval_dataset

(Dataset({
     features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 8000
 }),
 Dataset({
     features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 2000
 }))

In [ ]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # Output directory
    evaluation_strategy="epoch",    # Evaluate after every epoch
    learning_rate=5e-5,
    per_device_train_batch_size=4,  # Batch size
    num_train_epochs=3,             # Number of epochs
    save_strategy="epoch",
    logging_dir="./logs",           # Logging directory
    logging_steps=10,
    fp16=False                     # Set True if using a GPU with mixed precision
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset

)

# Train the model
trainer.train()



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jrvazhiyil (jrvazhiyil-fanshawe-college). Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.063600,0.052493
2,0.044500,0.042262
3,0.088000,0.034730


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou

TrainOutput(global_step=6000, training_loss=0.6059545395473639, metrics={'train_runtime': 1111.5549, 'train_samples_per_second': 21.591, 'train_steps_per_second': 5.398, 'total_flos': 1118889050112000.0, 'train_loss': 0.6059545395473639, 'epoch': 3.0})

In [ ]:
#model's state has changed after finetuning, save this state
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/spiece.model',
 './fine_tuned_model/added_tokens.json')

Testing


In [ ]:
#make sure prompts are always like this to get recommnedations
prompt = " Suggest some reccomnedation for the partient with these traits, Anemia: 1, Age: 22, Sex: 1 Creatinine: 2593, Diabetes: 1, Ejection Fraction: 22.2, High Blood Pressure: 1, Platelets: 120390, Serum Creatinine: 6, Serum Sodium: 126, Smoking: 1, Follow-up Time: 2, Dead: 1"


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/fine_tuned_model")

base_model = AutoModelForSeq2SeqLM.from_pretrained(
    "/content/fine_tuned_model",
    device_map="auto",
    torch_dtype="auto",
)


# Load the LoRA configuration because we used lora
model = PeftModel.from_pretrained(base_model, "/content/fine_tuned_model")



You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [ ]:
model.eval()
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate a response
outputs = model.generate(
    input_ids=inputs["input_ids"],
    max_length=120,          # Maximum length of the generated response
    temperature=0.5,         # Sampling temperature (lower = more focused response)
    top_p=0.8,               # Top-p nucleus sampling
    repetition_penalty=1.1,  # Penalty for repeated tokens
    do_sample=True           # Enable sampling for diverse outputs
)

# Decode the generated output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("AI Response:", response)

AI Response: Consult nephrology for renal management and consider dialysis if symptoms of uremia develop. Intensify blood pressure control with ACE inhibitors or ARBs. Initiate smoking cessation interventions. Treat anemia with appropriate iron supplements or erythropoiesis-stimulating agents. Optimize glycemic control with appropriate antidiabetic therapy. Discuss advanced heart failure therapies and palliative care options if prognosis worsens.


In [ ]:
import shutil
import os

def download_folder(folder_path, zip_filename):

  # Create a zip file of the folder
  shutil.make_archive(zip_filename, 'zip', folder_path)

  # Download the zip file
  from google.colab import files
  files.download(zip_filename + '.zip')

# Example usage:
folder_to_download = '/content/fine_tuned_model'  # Replace with your folder path
zip_file_name = 'fine_tuned_model'  # Replace with your desired zip file name

download_folder(folder_to_download, zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

_______________________________________

# Classification Model

In [ ]:
!pip install kaggle
!kaggle datasets download andrewmvd/heart-failure-clinical-data
!unzip heart-failure-clinical-data.zip

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data
License(s): Attribution 4.0 International (CC BY 4.0)
  0% 0.00/3.97k [00:00<?, ?B/s]
100% 3.97k/3.97k [00:00<00:00, 3.99MB/s]
Archive:  heart-failure-clinical-data.zip
  inflating: heart_failure_clinical_records_dataset.csv  


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

df = pd.read_csv('heart_failure_clinical_records_dataset.csv')
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [ ]:
df.columns

Index(['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time',
       'DEATH_EVENT'],
      dtype='object')

In [ ]:
#prepare the feature and target
X = df[['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']]
y = df['DEATH_EVENT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # 80% train, 20% test


In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [ ]:
y_pred = model.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8535564853556485


In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7833333333333333


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(model, param_grid, cv=5, scoring ='accuracy')


# Fit the grid search to the data
grid_search.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which 

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
             param_grid={'C': [0.1, 1, 10],
                         'max_iter': [100, 200, 300, 400, 500, 600, 700, 800,
                                      900, 1000],
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']},
             scoring='accuracy')

In [ ]:
# Get the best hyperparameters
best_params = grid_search.best_params_
print(f"Best hyperparameters: {best_params}")

# Train the model with the best hyperparameters
best_model = LogisticRegression(**best_params)
best_model.fit(X_train, y_train)

# Evaluate the model

y_pred_train = best_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred_train)

y_pred_test = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)

print(f"Train Accuracy with best hyperparameters: {train_accuracy}")
print(f"Test Accuracy with best hyperparameters: {test_accuracy}")

Best hyperparameters: {'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Train Accuracy with best hyperparameters: 0.8702928870292888
Test Accuracy with best hyperparameters: 0.75


____________________________________________

# Inference

In [ ]:
# pick a specific sample
sample = X_test.iloc[3]

In [ ]:
sample

,9
age,80.0
anaemia,1.0
creatinine_phosphokinase,123.0
diabetes,0.0
ejection_fraction,35.0
high_blood_pressure,1.0
platelets,388000.0
serum_creatinine,9.4
serum_sodium,133.0
sex,1.0


In [ ]:
#use the model to predict if the perosn is about to die
result = model.predict(sample.values.reshape(1, -1))
result.item() #this one will be in our prompt

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


1

In [ ]:
#convert our data into prompt
def convert_to_prompt(row):
  input_str = (
        f"Anemia: {row['anaemia']}, Age: {row['age']}, Sex: {row['sex']}, Creatinine: {row['creatinine_phosphokinase']}, "
        f"Diabetes: {row['diabetes']}, Ejection Fraction: {row['ejection_fraction']}, "
        f"High Blood Pressure: {row['high_blood_pressure']}, Platelets: {row['platelets']}, "
        f"Serum Creatinine: {row['serum_creatinine']}, Serum Sodium: {row['serum_sodium']}, "
        f"Smoking: {row['smoking']}, Follow-up Time: {row['time']}, Dead: {result.item()}"
    )
  return input_str



In [ ]:
converted_prompt = convert_to_prompt(sample)
converted_prompt

'Anemia: 1.0, Age: 80.0, Sex: 1.0, Creatinine: 123.0, Diabetes: 0.0, Ejection Fraction: 35.0, High Blood Pressure: 1.0, Platelets: 388000.0, Serum Creatinine: 9.4, Serum Sodium: 133.0, Smoking: 1.0, Follow-up Time: 10.0, Dead: 1'

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/fine_tuned_model")

base_model = AutoModelForSeq2SeqLM.from_pretrained(
    "/content/fine_tuned_model",
    device_map="auto",
    torch_dtype="auto",
)


# Load the LoRA configuration because we used lora
llm = PeftModel.from_pretrained(base_model, "/content/fine_tuned_model")

llm.eval()
inputs = tokenizer(converted_prompt, return_tensors="pt").to(llm.device)

# Generate a response
outputs = llm.generate(
    input_ids=inputs["input_ids"],
    max_length=120,          # Maximum length of the generated response
    temperature=0.5,         # Sampling temperature (lower = more focused response)
    top_p=0.8,               # Top-p nucleus sampling
    repetition_penalty=1.1,  # Penalty for repeated tokens
    do_sample=True           # Enable sampling for diverse outputs
)

# Decode the generated output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("AI Response:", response)

AI Response: Consult nephrology for renal management and consider dialysis if symptoms of uremia develop. Carefully correct serum sodium under close monitoring. Discuss advanced heart failure therapies and palliative care options if prognosis worsens.


#Evaluation

The BLEU (Bilingual Evaluation Understudy) score is a metric used to evaluate the quality of machine-generated text, especially in machine translation tasks. It measures the similarity between the generated text and one or more reference texts.

The BLEU score provides a quantitative measure of how well the generated recommendations align with the expected recommendations or guidelines for a given patient's condition.

By comparing the generated recommendations to a reference set of recommendations, you can assess the accuracy and relevance of the recommendations produced by your model.


In [ ]:
df2.iloc[0]['input']

'Anemia: 0, Age: 64, Sex: 1, Creatinine: 3230.2, Diabetes: 1, Ejection Fraction: 31.2, High Blood Pressure: 0, Platelets: 263119.9, Serum Creatinine: 5.3, Serum Sodium: 129.7, Smoking: 1, Follow-up Time: 15, Dead: 1'

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
llm.eval()
inputs = tokenizer(df2.iloc[0]['input'], return_tensors="pt").to(llm.device)

# Generate a response
outputs = llm.generate(
    input_ids=inputs["input_ids"],
    max_length=120,          # Maximum length of the generated response
    temperature=0.5,         # Sampling temperature (lower = more focused response)
    top_p=0.8,               # Top-p nucleus sampling
    repetition_penalty=1.1,  # Penalty for repeated tokens
    do_sample=True           # Enable sampling for diverse outputs
)

# Decode the generated output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

#caluclate bleu score
from nltk.translate.bleu_score import sentence_bleu
candidate = response.split() #genrated
reference = [[word for word in df2.iloc[0]['target'].split()]]   #ground truth
score = sentence_bleu(reference, candidate)
print(f"BLEU Score: {score}")

BLEU Score: 0.6278303582831738


Indicates moderate performance.